In [1]:
!pip install gymnasium
!pip install stable-baselines3==2.0.0 
!pip install numpy==1.24.4
!pip install swig
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #this line should change for mac, you can find it on pytorch
!pip install tensorflow
%load_ext tensorboard
import tensorflow as tf
import datetime

  Using cached stable_baselines3-2.0.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached gymnasium-0.28.1-py3-none-any.whl.metadata (9.2 kB)
Using cached stable_baselines3-2.0.0-py3-none-any.whl (178 kB)
Using cached gymnasium-0.28.1-py3-none-any.whl (925 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.1.1
    Uninstalling gymnasium-1.1.1:
      Successfully uninstalled gymnasium-1.1.1
  Attempting uninstall: stable-baselines3
    Found existing installation: stable_baselines3 2.6.0
    Uninstalling stable_baselines3-2.6.0:
      Successfully uninstalled stable_baselines3-2.6.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sb3-contrib 2.6.0 requires stable_baselines3<3.0,>=2.6.0, but you have stable-baselines3 2.0.0 which is incompatible.


  Using cached numpy-1.24.4.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\irmak\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\irmak\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\irmak\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\irmak\anaconda3\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
      obj = import_module(mod_path)
            ^^^^^^^^^^^^^^^^^^^^^

ERROR: Invalid requirement: '#this': Expected package name at the start of dependency specifier
    #this
    ^


In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...


In [3]:
import json
import sys
import numpy as np
import requests
import gymnasium as gym

In [4]:
sys.path.insert(0,'boptestGymService')
from boptestGymEnv import BoptestGymEnv

# url for the BOPTEST service
url = 'https://api.boptest.net'
TEST_CASE = "bestest_hydronic_heat_pump"

C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:122: SyntaxWarning: invalid escape sequence '\c'
  axs[0].set_ylabel('Operative\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:131: SyntaxWarning: invalid escape sequence '\c'
  axs[3].set_ylabel('Ambient\ntemperature\n($^\circ$C)')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:135: SyntaxWarning: invalid escape sequence '\d'
  axt.plot(x_time, df['weaSta_reaWeaHDirNor_y'], color='gold', linestyle='-', linewidth=1, label='$\dot{Q}_rad$')
C:\Users\irmak\Documents\GitHub\RL_ex\boptestGymService\examples\test_and_plot.py:141: SyntaxWarning: invalid escape sequence '\d'
  axs[3].plot([],[], color='gold',        linestyle='-', linewidth=1, label='$\dot{Q}_{rad}$')


In [ ]:
# Instantiate environment
env = BoptestGymEnv(url                   = url,
                    testcase              = 'bestest_hydronic_heat_pump',
                    actions               = ['oveHeaPumY_u'],
                    observations          = {'reaTZon_y':(280.,310.)},
                    random_start_time     = False,
                    start_time            = 31*24*3600,
                    max_episode_length    = 24*3600,
                    warmup_period         = 0,
                    step_period           = 3600)

In [ ]:
env?

In [ ]:
obs, _ = env.reset()
print('Zone temperature: {:.2f} degC'.format(obs[0]-273.15))
print('Episode starting day: {:.1f} (from beginning of the year)'.format(env.start_time/24/3600))

In [ ]:
print('Observation space of the building environment:')
print(env.observation_space)
print('Action space of the building environment:')
print(env.action_space)

In [ ]:
print(env)

In [ ]:
from boptestGymEnv import DiscretizedActionWrapper
env = DiscretizedActionWrapper(env,n_bins_act=1)
print('Action space of the wrapped agent:')
print(env.action_space)
print('Action space of the original agent:')
print(env.unwrapped.action_space)

env.unwrapped.stop()

In [ ]:
#env.stop()

In [ ]:
obs, _ = env.reset()

In [ ]:
class Q_Learning_Agent(object):

  def __init__(self, env, eps_min=0.01, eps_decay=0.01, alpha=0.05, gamma=0.9):
    '''Constructor of a q-learning agent. Assumes discrete state and action spaces.

    '''
    self.env       = env
    self.eps_min   = eps_min
    self.eps_decay = eps_decay
    self.alpha     = alpha
    self.gamma     = gamma

    # Initialize epsilon
    self.eps       = 1.0

    # Initialize q-function as a null function
    self.q = np.zeros((env.observation_space.n,
                       env.action_space.n))

  def predict(self, obs, deterministic=True):
    '''Method to select an action with an epsilon-greedy policy.

    '''
    if deterministic:
      # Use q-function to decide action
      return np.argmax(self.q[obs])
    else:
      if self.eps > self.eps_min:
        # Linearly decreasing schedule
        self.eps -= self.eps_decay
      if np.random.random() < self.eps:
        # Explore with random action
        return np.random.choice([a for a in range(env.action_space.n)])
      else:
        # Exploit the information of our q-function
        return np.argmax(self.q[obs])

  def learn(self, total_episodes=10):
    '''Learn from a number of interactions with the environment.

    '''
    for i in range(total_episodes):
      # Initialize enviornment
      done = False
      obs, _  = env.reset()
      # Print episode number and starting day from beginning of the year:
      print('-------------------------------------------------------------------')
      print('Episode number: {0}, starting day: {1:.1f} ' \
            '(from beginning of the year)'.format(i+1, env.start_time/24/3600))

      while not done:
        # Get action with epsilon-greedy policy and simulate
        act                   = self.predict(obs, deterministic=False)
        nxt_obs, rew, terminated, truncated, _ = env.step(act)
        done = (terminated or truncated)
        # Compute temporal difference target and error to udpate q-function
        td_target         = rew + self.gamma*np.max(self.q[nxt_obs])
        td_error          = td_target - self.q[obs][act]
        self.q[obs][act] += self.alpha*td_error
        # Make our next observation the current observation
        obs = nxt_obs
      # Print the q-function after every episode to show progress
      print('q(s,a) = ')
      print(self.q)

In [ ]:
# Redefine reward function
class BoptestGymEnvCustomReward(BoptestGymEnv):
    '''Define a custom reward for this building

    '''
    def get_reward(self):
        '''Custom reward function. To expedite learning, we use a clipped reward
        function that has a value of 1 when there is no increase in discomfort
        and 0 otherwise. We use the BOPTEST `GET /kpis` API call to compute the
        total cummulative discomfort from the beginning of the episode. Note
        that this is the true value that BOPTEST uses when evaluating
        controllers.

        '''
        # Compute BOPTEST core kpis
        kpis = requests.get('{0}/kpi/{1}'.format(self.url, self.testid)).json()['payload']
        # Calculate objective integrand function as the total discomfort
        objective_integrand = kpis['tdis_tot']
        # Give reward if there is not immediate increment in discomfort
        if objective_integrand == self.objective_integrand:
          reward=1
        else:
          reward=0
        # Record current objective integrand for next evaluation
        self.objective_integrand = objective_integrand
        return reward

In [ ]:
env.stop()
import random

# Seed for random starting times of episodes
seed = 123456
random.seed(seed)
# Seed for random exploration and epsilon-greedy schedule
np.random.seed(seed)

# Winter period goes from December 21 (day 355) to March 20 (day 79)
excluding_periods = [(79*24*3600, 355*24*3600)]
# Temperature setpoints
lower_setp = 21 + 273.15
upper_setp = 24 + 273.15
# Instantiate environment
env = BoptestGymEnvCustomReward(url                   = url,
                                testcase              = 'bestest_hydronic_heat_pump',
                                actions               = ['oveHeaPumY_u'],
                                observations          = {'reaTZon_y':(lower_setp,upper_setp)},
                                random_start_time     = False,
                                start_time            = 31*24*3600,
                                excluding_periods     = excluding_periods,
                                max_episode_length    = 2*24*3600,
                                warmup_period         = 24*3600,
                                step_period           = 3600,
                                render_episodes       = True)

In [ ]:
from boptestGymEnv import DiscretizedObservationWrapper
from boptestGymEnv import DiscretizedActionWrapper

In [ ]:
env = DiscretizedActionWrapper(env, n_bins_act=1)
env = DiscretizedObservationWrapper(env, n_bins_obs=3, outs_are_bins=True)

In [ ]:
print('Observation space of the building environment:')
print(env.observation_space.sample())
print(env.observation_space)
print('Action space of the building environment:')
print(env.action_space)
print(env.action_space.sample())

In [ ]:
model = Q_Learning_Agent(env, eps_min=0.01, eps_decay=0.001, alpha=0.1, gamma=0.9)

In [ ]:
model.learn(total_episodes=10)

In [ ]:
import matplotlib.pyplot as plt

acts   = ['a=0','a=1']
stas   = ['T<21', '21<T<24', 'T>24']
colors = ['b',    'g',       'r']

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('actions',           labelpad=6,  fontsize=12)
ax.set_ylabel('states',            labelpad=10, fontsize=12)
ax.set_zlabel('$\mathbf{q(s,a)}$', labelpad=0,  fontsize=15)
plt.xticks(ticks=range(len(acts)), labels=acts)
plt.yticks(ticks=range(len(stas)), labels=stas)

for i, s in enumerate(stas):
  x = np.arange(len(acts))
  h = model.q[i,:]

  # Set color
  color = [colors[i]]*len(acts)

  # Plot the 3D bar graph
  ax.bar(x, h, zs=i, zdir='y', color=color, alpha=0.8)

plt.show()

In [ ]:
# Compute the state-value function
v = np.amax(model.q, axis=1)

# Plot state-value function
fig = plt.figure()

ax = fig.add_subplot(111)
ax.set_xlabel('states', labelpad=10, fontsize=12)
ax.set_ylabel('$\mathbf{v(s)}$', labelpad=0,  fontsize=15)
plt.xticks(ticks=range(len(stas)), labels=stas)
x = np.arange(len(stas))
ax.bar(x, v, color=colors, alpha=0.8)
plt.show()

In [ ]:
env.stop()
env = BoptestGymEnvCustomReward(url                   = url,
                                testcase              = 'bestest_hydronic_heat_pump',
                                actions               = ['oveHeaPumY_u'],
                                observations          = {'reaTZon_y':(lower_setp,upper_setp)},
                                random_start_time     = False,
                                start_time            = 31*24*3600,
                                max_episode_length    = 24*3600,
                                warmup_period         = 24*3600,
                                step_period           = 3600)
env = DiscretizedActionWrapper(env, n_bins_act=1)
env = DiscretizedObservationWrapper(env, n_bins_obs=3, outs_are_bins=True)

done = False
obs, _ = env.reset()

from IPython.display import clear_output
while not done:
  # Clear the display output at each step
  clear_output(wait=True)
  # Compute control signal
  action = model.predict(obs, deterministic=True)
  # Print the current operative temperature and decided action
  print('-------------------------------------------------------------------')
  print('State  [Bin #]  = {:.0f}'.format(obs))
  print('Action [ - ]    = {:.0f}'.format(action))
  print('-------------------------------------------------------------------')
  # Implement action
  obs,reward,terminated,truncated,info = env.step(action) # send the action to the environment
  done = (terminated or truncated)

In [ ]:
env.get_kpis()